<a href="https://colab.research.google.com/github/TATENDA-MAX/NLP-Summarizing-PDFs/blob/main/PDF_Summarizer_Deep_Learning_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import os
import textwrap
import warnings

# 1. Check for PyTorch immediately
try:
    import torch
except ImportError:
    print("\nCRITICAL ERROR: PyTorch is not installed.")
    print("The deep learning model requires PyTorch to run.")
    print("Please paste this into your terminal to fix it:")
    print("    pip install torch torchvision torchaudio\n")
    sys.exit(1)

import fitz  # PyMuPDF

# Fallback Library for broken PDFs
try:
    import pypdf
except ImportError:
    pypdf = None

# Filter out the annoying "IProgress not found" warning from TQDM
warnings.filterwarnings("ignore", category=UserWarning, module='tqdm')

try:
    from transformers import pipeline
except ImportError:
    print("\nError: Transformers library not found.")
    print("Please run: pip install transformers")
    sys.exit(1)

def extract_text_from_pdf(pdf_path):
    """
    Extracts text from a PDF file using PyMuPDF.
    """
    if not os.path.exists(pdf_path):
        print(f"Error: File not found - {pdf_path}")
        return None

    text = ""

    try:
        doc = fitz.open(pdf_path)
        for page in doc:
          try:
            # Try to get text from page
            page_text = page.get_text()
            text += page.get_text() + "\n"
          except Exception:
            # If a specific page is corrupt, skip it and continue
            continue

    except Exception as e:
        print(f"Warning: PyMuPDF encountered corruption ({e}). Switching to fallback...")


    # Method 2: pypdf (Slower, but robust against corruption)
    # Use this if PyMuPDF failed or returned no text
    if not text.strip():
      if pypdf:
        print("   - Attempting extraction with pypdf (more robust)...")
        try:
          reader = pypdf.PdfReader(pdf_path)
          # Iterate pages and extract text
          for page in reader.pages:
            extracted = page.extract_text()
            if extracted:
              text += extracted + "\n"
          print("  -Extraction successful with pypdf.")
        except Exception as e:
          print(f"   - Error extracting text with pypdf: {e}")
      else:
        print(" - PDF is corrupted and pypdf is not installed.")
        print(" - Please install pypdf to use this method.")
        return None

    return text
def chunk_text(text, max_chunk_size=2000):
    """
    Splits text into chunks to respect the model's input token limit.
    """
    chunks = textwrap.wrap(text, width=max_chunk_size, break_long_words=False, replace_whitespace=False)
    return chunks

def summarize_pdf(pdf_path):
    """
    Main function to process the PDF and generate a summary.
    """
    # OPTIMIZATION: Use a distilled model for faster inference speed (2x-3x faster)
    # Original (Slower, higher quality): "facebook/bart-large-cnn"
    # New (Faster, good quality): "sshleifer/distilbart-cnn-12-6"
    model_name = "sshleifer/distilbart-cnn-12-6"

    print(f"1. Loading Deep Learning Model ({model_name})...")

    # Debug: Confirm PyTorch is active
    try:
        print(f"   - PyTorch detected: Version {torch.__version__}")
    except Exception:
        print("   - Warning: Could not detect PyTorch version.")

    # OPTIMIZATION: Check for GPU
    if torch.cuda.is_available():
        device = 0
        print("   - GPU DETECTED: Running on generic GPU (Fast).")
    else:
        device = -1
        print("   - NO GPU DETECTED: Running on CPU (Slow).")

    try:
        # Initialize the summarization pipeline
        # framework="pt" forces PyTorch (fixes 'name torch not defined' error)
        summarizer = pipeline(
            "summarization",
            model=model_name,
            device=device,
            framework="pt"
        )
    except Exception as e:
        print(f"\nError loading model: {e}")
        if "torch" in str(e):
            print("\nPOSSIBLE FIX: The Transformers library is failing to find PyTorch.")
            print("Please try running this command to upgrade the libraries:")
            print("    pip install --upgrade transformers torch")
        else:
            print("Ensure you have an internet connection to download the model.")
        return

    print(f"2. Extracting text from {pdf_path}...")
    text = extract_text_from_pdf(pdf_path)
    if not text:
        return

    print("3. Chunking text...")
    chunks = chunk_text(text, max_chunk_size=2000)
    print(f"   - Split document into {len(chunks)} chunks.")

    print("4. Generating Summaries (this may take time)...")
    final_summary = []

    for i, chunk in enumerate(chunks):
        # Skip empty chunks
        if len(chunk.strip()) < 50:
            continue

        print(f"   - Processing chunk {i+1}/{len(chunks)}...")

        try:
            # BART parameters:
            # max_length: max tokens for the summary output
            # min_length: min tokens for the summary output
            summary_output = summarizer(chunk, max_length=150, min_length=40, do_sample=False, truncation=True)
            summary_text = summary_output[0]['summary_text']
            final_summary.append(summary_text)
        except Exception as e:
            print(f"   - Error processing chunk {i+1}: {e}")
            continue

    full_summary = " ".join(final_summary)

    print("\n" + "="*50)
    print("FINAL DOCUMENT SUMMARY")
    print("="*50)
    print(full_summary)
    print("="*50)

    # Optional: Save to file
    with open("summary.txt", "w", encoding="utf-8") as f:
        f.write(full_summary)
    print("Summary saved to 'summary.txt'")

if __name__ == "__main__":
    if len(sys.argv) > 1:
        input_pdf = "/content/merged_deep_learning_notes_3-75.pdf"
        summarize_pdf(input_pdf)
    else:
        print("Usage: python pdf_summarizer.py <path_to_pdf>")
        print("Please provide a PDF file path.")

1. Loading Deep Learning Model (sshleifer/distilbart-cnn-12-6)...
   - PyTorch detected: Version 2.9.0+cu126
   - GPU DETECTED: Running on generic GPU (Fast).


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0


2. Extracting text from /content/merged_deep_learning_notes_3-75.pdf...
3. Chunking text...
   - Split document into 1761 chunks.
4. Generating Summaries (this may take time)...
   - Processing chunk 1/1761...
   - Processing chunk 2/1761...
   - Processing chunk 3/1761...
   - Processing chunk 4/1761...
   - Processing chunk 5/1761...
   - Processing chunk 6/1761...
   - Processing chunk 7/1761...
   - Processing chunk 8/1761...
   - Processing chunk 9/1761...
   - Processing chunk 10/1761...


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


   - Processing chunk 11/1761...
   - Processing chunk 12/1761...
   - Processing chunk 13/1761...
   - Processing chunk 14/1761...
   - Processing chunk 15/1761...


Token indices sequence length is longer than the specified maximum sequence length for this model (1054 > 1024). Running this sequence through the model will result in indexing errors


Streaming output truncated to the last 5000 lines.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

   - Processing chunk 1049/1761...
   - Error processing chunk 1049: CUDA error: device-side assert triggered
Search for `cudaErrorAssert' in https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__TYPES.html for more information.
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.

   - Processing chunk 1050/1761...
   - Error processing chunk 1050: CUDA error: device-side assert triggered
Search for `cudaErrorAssert' in https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__TYPES.html for more information.
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BL

In [1]:
!pip install pypdf
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.5/329.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 74.7 MB/s eta 0:00:00
